In [1]:
import os ##importing modules os -> accesing files
import numpy as np ## -> create arrays for image after read
import pandas as pd ## -> extracting information needed from the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
basepath = "/content/drive/MyDrive"
filename = "archive.zip"

In [ ]:
import zipfile
with zipfile.ZipFile(os.path.join(basepath,filename), 'r') as zipObject: ##unzip dataset
  zipObject.extractall(path=os.path.join(basepath, ""))

In [ ]:
mass_train_metadata = pd.read_csv(os.path.join(basepath, "csv", "mass_case_description_train_set.csv")) ##mass_train dataset

In [8]:
mass_train_metadata.drop_duplicates(subset="patient_id", keep='first', inplace=True)
mass_train_metadata.head() ##removes duplicates (one patient could have multiple images)

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...
5,P_00009,3,RIGHT,CC,1,mass,OVAL,ILL_DEFINED,4,MALIGNANT,4,Mass-Training_P_00009_RIGHT_CC/1.3.6.1.4.1.959...,Mass-Training_P_00009_RIGHT_CC_1/1.3.6.1.4.1.9...,Mass-Training_P_00009_RIGHT_CC_1/1.3.6.1.4.1.9...
7,P_00015,3,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED-SPICULATED,4,MALIGNANT,4,Mass-Training_P_00015_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00015_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00015_LEFT_MLO_1/1.3.6.1.4.1.9...
8,P_00018,2,RIGHT,CC,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00018_RIGHT_CC/1.3.6.1.4.1.959...,Mass-Training_P_00018_RIGHT_CC_1/1.3.6.1.4.1.9...,Mass-Training_P_00018_RIGHT_CC_1/1.3.6.1.4.1.9...


In [20]:
def get_image_file(): ##utility function to get image file path sub-dataset
  train_data = []
  for rows in mass_train_metadata.itertuples():
    id = rows.patient_id
    corresponding_row = mass_train_metadata[mass_train_metadata.patient_id == id]
    corresponding_row = corresponding_row[['patient_id', 'image file path', 'pathology']]
    train_data.append(corresponding_row)
  return train_data

In [64]:
train_data = get_image_file() ##train_data now includes, patient_id, the file path, and label

,patient_id,image file path,pathology
0,P_00001,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,MALIGNANT


In [68]:
for x in range (len(train_data)):   ## get the image directory
  z = train_data[x]['image file path'].str.split("/")
  z = list(z)
  img_filename = z[0][2]
  train_data[x]['img_file'] = img_filename

In [76]:
def replace_label(train_data): ##replace label from string into integers
  for x in range(len(train_data)):
    if(train_data[x]['pathology'].item() == "MALIGNANT"):
      train_data[x]['pathology'] = int(2)
    elif(train_data[x]['pathology'].item() == "BENIGN"):
      train_data[x]['pathology'] = int(0)
    else:
      train_data[x]['pathology'] = int(1)

In [77]:
replace_label(train_data)

In [79]:
import cv2 as cv

In [119]:
def process_img(data): ##read image using openCV
  dataset = []
  for x in range(len(data)):
    img_folder = data[x]['img_file'].item()
    img_label = data[x]['pathology'].item()
    img_file_path = os.path.join(basepath, "jpeg", img_folder)
    img_file_path = os.path.join(img_file_path, os.listdir(img_file_path)[0])
    image = cv.imread(img_file_path)

    try:
      resized = cv.resize(image, (128,128)) ## resize image into 128x128
      dataset.append([resized, img_label])
    except Exception as e:
      print(e)
  return dataset


In [96]:
train_img_label = process_img(train_data)

In [98]:
from random import shuffle ##shuffle the image dataset
shuffle(train_img_label)

In [101]:
train_data = []
train_label = []
for data,label in train_img_label:
  train_data.append(data) ## append image to one list, and the label to the other (later on used to create the tensors)
  train_label.append(label)

In [111]:
train_data = np.array(train_data) / 255 ##normalization
train_label = np.array(train_label)

In [107]:
import tensorflow as tf
keras = tf.keras
train_label = keras.utils.to_categorical(train_label) ##turns label to one-hot encoding
print(train_label)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [113]:
np.save('traindata', train_data) ## save the data
np.save('trainlabel', train_label)

Code below does the same procedure except for the test dataset

In [ ]:
mass_test_metadata = pd.read_csv(os.path.join(basepath, "csv", "mass_case_description_test_set.csv"))
mass_test_metadata.drop_duplicates(subset="patient_id", keep='first', inplace=True)
mass_test_metadata.head()

In [116]:
test_data = []
for rows in mass_test_metadata.itertuples():
    id = rows.patient_id
    corresponding_row = mass_test_metadata[mass_test_metadata.patient_id == id]
    corresponding_row = corresponding_row[['patient_id', 'image file path', 'pathology']]
    test_data.append(corresponding_row)

In [ ]:
replace_label(test_data)

In [122]:
for x in range (len(test_data)):   
  z = test_data[x]['image file path'].str.split("/")
  z = list(z)
  img_filename = z[0][2]
  test_data[x]['img_file'] = img_filename

In [123]:
test_img_label = process_img(test_data)

In [124]:
shuffle(test_img_label)

In [125]:
test_data = []
test_label = []
for data,label in train_img_label:
  test_data.append(data)
  test_label.append(label)

In [126]:
test_data = np.array(test_data) / 255
test_label = np.array(test_label)

In [ ]:
test_label = keras.utils.to_categorical(test_label)
print(test_label)

In [130]:
np.save('testdata', test_data)
np.save('testlabel', test_label)